# Merging Datasets
# 02_EDA_merging_files

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 22/09/2025 | Adrienne | Updated | Made significant changes to program | 
| 23/09/2025 | Adrienne | Updated | Restructured file to merge EOB and Claim |
| 28/09/2025 | Adrienne | Updated | Moved Martin's code to program |
| 13/10/2025 | Martin | Updated | Code cleanup |

# Content

* [Summary](#summary)
* [Merging Datasets](#merging-datasets)


# Summary 
Through EDA we have discovered that the coverage and claim response files do not contain any information that would be useful for our unsupervised learning task. Furthermore, the Patient file also seems to be limited in the number of records we can pull from the API. This program attempts to merge some of the files to see if there are overlapping patients in both files.

Some Notes on Idenfifiers:
- EOB does not have Patient Medicare Number
- Claim does not have Patient Number
- Coverage (subscriberId) -> (Many-to-One) -> Patient (patient_medicare_number)
- Not all have a value, but if there is, 4 values per patient (Part A-D)

File Relationships
- EOB (patient) -> (Many-to-One) -> Patient (id)
        Need to convert EOB col to int
        Only ~50 patients with information
- Claim (patient_medicare_number) -> (Many-to-One) -> Patient (identifier_1_value)
- Claim Response (id) -> (One-to-One) -> Claim (id)

Findings

- Claim Response has One-to-One relationship with Claim, but doesn't really provide any additional info
- Seems like only diagnosis, priority, claim-type total seem to be the most "relevant" columns

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines

In [ ]:
# readin clean datafiles
path = "../data/clean/"
# eob_df = pd.read_pickle(path + 'explanation_of_benefit_clean.pkl')
eob_df = pd.read_pickle(path + 'eob.pkl')
# coverage_df = pd.read_pickle(path + 'coverage.pkl')
claim_df = pd.read_pickle(path + 'claim_sample.pkl')
# claim_df =  pd.read_pickle(path + 'claim.pkl')
# claim_response_df =  pd.read_pickle(path + 'claim_response.pkl')
patient_df = pd.read_pickle(path + 'patient.pkl')

Creating some common variables across datafiles

In [4]:
# Preprocess identifiers in EOB
eob_df['patient_medicare_number'] = eob_df['contained_0_identifier_0_value']
eob_df['patient_number'] = eob_df['id'].str.split('-').str[-1]
#eob_df['unique_claim_ID'] = eob_df['claimId_2'].str.replace(r'[-]', '', regex=True)

In [ ]:
# needed to convert beneficiary to string to merge on
coverage_df['patient_number'] = coverage_df['beneficiary'].apply(str)

In [5]:
# Preprocess Identifier in Patient
patient_df['patient_medicare_number'] = patient_df['identifier_1_value']
patient_df['patient_number'] = patient_df['identifier_0_value'].str.replace(r'[-]', '', regex=True)
patient_df['patient_first_name'] = patient_df['name_0_given'].str.replace(r'[ \[ \]"]', '', regex=True)
patient_df['patient_last_name'] = patient_df['name_0_family']

Checking unique values and column lengths for merging

In [ ]:
# EOB
print(f"Number of unique patient numbers in EOB: {len(eob_df['patient_number'].unique())}")
# print(f"Number of unique claim IDs in EOB: {len(eob_df['unique_claim_ID'].unique())}")

# Patient
print(f"Number of unique patient numbers in Patient: {len(patient_df['patient_number'].unique())}")
print(f"Number of unique patient medicare numbers in Patient: {len(patient_df['patient_medicare_number'].unique())}")
print(f"Number of unique patient first names in Patient: {len(patient_df['patient_first_name'].unique())}")

# Coverage
# print(f"Number of unique patient numbers in Coverage: {len(coverage_df['patient_number'].unique())}")

# Claim
print(f"Number of unique patient medicare numbers in Claim: {len(claim_df['patient_medicare_number'].unique())}")
print(f"Number of unique claim IDs in Claim: {len(claim_df['unique_claim_ID'].unique())}")
# print(f"Number of unique patient medicare numbers in Claim Response: {len(claim_response_df['patient_medicare_number'].unique())}")

num of unique patient numbers in EOB: 3233
num of unique patient numbers in Patient: 5000
num of unique patient medicare numbers in Patient: 5000
num of unique patient first names in Patient: 2
num of unique patient medicare numbers in Claim: 2707
num of unique claim IDs in Claim: 19145


Counts by column length

In [ ]:
# Checking lengths of values in columns
print(eob_df['patient_number'].str.len().unique())
# print(eob_df['unique_claim_ID'].str.len().unique())
print(patient_df['patient_number'].str.len().unique())
print(patient_df['patient_medicare_number'].str.len().unique())
print(claim_df['patient_medicare_number'].str.len().unique())
print(claim_df['unique_claim_ID'].str.len().unique())
print(claim_df['identifier_0_value'].str.len().unique())
# print(claim_response_df['patient_medicare_number'].str.len().unique())

[10 11  9  8  7]
[14]
[11]
[11]
[19  9 14]
[23 13 10 15]


In [ ]:
eob_df['length_counts'] = eob_df['patient_number'].str.len()
# claim_df['length_counts'] = claim_df['identifier_0_value'].str.len()
length_distribution = eob_df['length_counts'].value_counts()
length_distribution

length_counts
10    1914
11    1138
9      161
8       19
7        1
Name: count, dtype: int64

In [ ]:
claim_df['length_counts'] = claim_df['unique_claim_ID'].str.len()
# claim_df['length_counts'] = claim_df['identifier_0_value'].str.len()
length_distribution = claim_df['length_counts'].value_counts()
length_distribution

length_counts
9     17996
19     1167
14      837
Name: count, dtype: int64

# Merging files

Attempting to merge Patient and Explanation of Benefits datasets with the Claims dataset

In [ ]:
# For ease of merging initially limiting dataset columns
eob_df_part = eob_df[['patient_number' ]]
patient_df_part = patient_df[['patient_number', 'patient_medicare_number']]
claim_df_part = claim_df[['patient_medicare_number', 'unique_claim_ID']]
# claim_response_df_part = claim_response_df[['patient_medicare_number', 'unique_claim_ID']]
# covergae_df_part = coverage_df[['patient_number']]


Checking to see if there are identifiers found in each dataset

In [ ]:
l1 = set(eob_df['patient_number'].unique()).intersection(patient_df['patient_number'])

0

In [ ]:
df = pd.merge(eob_df_part, patient_df_part, how='outer', on='patient_number')

In [ ]:
df1 = pd.merge(eob_df_part, claim_df_part, how='outer', on='unique_claim_ID')

In [ ]:
df2 = pd.merge(coverage_df_part, patient_df_part, how='outer', on='patient_number')

In [ ]:
def find_common_patients(claim: pd.DataFrame, eob: pd.DataFrame, patient: pd.DataFrame):
  """Finds the common patient ids that contain data in all 3 datasets

  Args:
      claim (pd.DataFrame): claim dataset
      eob (pd.DataFrame): eob dataset
      patient (pd.DataFrame): patient dataset

  Returns:
      set: List of Patient IDs
  """
  # Patients that have claim
  l1 = set(claim['patient_medicare_number'].unique()).intersection(patient['patient_medicare_number'])

  # Patients that have an eob & claim info
  l2 = set(patient[patient['patient_medicare_number'].isin(l1)]['id']).intersection(set(eob['patient_number'].astype(int)))

  # Get the medicare numbers and patient ids
  mcare = set(patient[patient['id'].isin(l2)]['patient_medicare_number'])
  print(len(l1))

  print("Patient Medicare Numbers:")
  print(mcare)
  print()
  print("Patient IDs")
  print(l2)

In [14]:
# These are patients that have data across all the data sets
find_common_patients(claim_df, eob_df, patient_df)

0
Patient Medicare Numbers:
set()

Patient IDs
set()
